In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
! unzip NORMAL_test.zip 

In [ ]:
! unzip PNEUMONIA_test.zip

In [ ]:
!unzip PNEUMONIA.zip

In [ ]:
! unzip NORMAL.zip

In [ ]:
! unzip PNEUMONIA_train.zip

In [ ]:
! unzip NORMAL_train.zip  

In [ ]:
!git clone https://github.com/vb100/Pneumonia-X-Rays-of-Human-Lungs-AI-project

## Libraries 

In [ ]:
import numpy as np
import pandas as pd
import itertools
import os, stat, time
from os.path import dirname as up
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.metrics import categorical_crossentropy
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from vb100_utils import *
from shutil import copyfile
import shutil
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
"""
!pip install sklearn
!pip install -U matplotlib
!pip install Pillow
"""

## data pipelining 

In [ ]:
# CONSTANTS FOR DIRECTORIES
TRAIN_DIR = 'Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train'
VALID_DIR = 'Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/val'
TEST_DIR = 'Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/test'
l_DIRS = [TRAIN_DIR, VALID_DIR, TEST_DIR]
POSITIVE_CLASS = 'NORMAL'
ABSTRACT_CLASS = 'PNEUMONIA'

# CONSTANTS FOR IMAGE PARAMETERS
INPUT_W = 1200 # pixels
INPUT_H = 900  # pixels
DIVIDER = 3.6
INPUT_DIM = (int(INPUT_W/DIVIDER), int(INPUT_H/DIVIDER), 1)
BATCH_SIZE_TRAIN = 64
BATCH_SIZE_TEST = 64 
BATCH_SIZE_VALID = 16
NORMALIZER = 1./255
IMAGE_FORMAT = 'jpeg'
# Output Info
print('Image dimmensions for CNN = {}'.format(INPUT_DIM))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
train_normal=os.listdir('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train/NORMAL/')

plt.suptitle('Normal Lungs')
for i in range(4,7):
    img =train_normal[i]
    #img = str(img)
    img = load_img('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train/NORMAL/'+img)
    plt.imshow(img)
    plt.show()

In [ ]:
import seaborn as sns
train_pneumonia=os.listdir('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train/BACTERIA/')
sns.barplot(x=['Normal','Pneumonia'],y=[len(train_normal),len(train_pneumonia)])

In [ ]:
if abstract_class_exists(ABSTRACT_CLASS, l_DIRS):
  
    structure_origin_data(l_DIRS, IMAGE_FORMAT, POSITIVE_CLASS)

classes = classes_for_each_set(l_DIRS)

print('Catched classes for the model:\n{}'.format(classes))

In [ ]:
# Generating and Plot Image Data from Train Set
TRAIN_BATCHES = ImageDataGenerator(rescale=NORMALIZER).\
    flow_from_directory(TRAIN_DIR,
    color_mode='grayscale',
    target_size=INPUT_DIM[0:2],
    classes=classes['TRAIN'],
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE_TRAIN)

imgs, labels = next(TRAIN_BATCHES)  # <-- Extracting image matrixes and labels
plots(imgs, titles=labels)          # <-- Plot Images with labels
#train_imgs = rgb_to_grayscale(imgs) # <-- Convert RGB images to Grayscale ones by Tensorflow
#train_labels = labels
# Generating and Plot Image Data from Test Set
TEST_BATCHES = ImageDataGenerator(rescale=NORMALIZER).\
    flow_from_directory(TEST_DIR,
    color_mode='grayscale',
    target_size=INPUT_DIM[0:2],
    classes=classes['TEST'],
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE_TEST)

imgs, labels = next(TEST_BATCHES)   # <-- Extracting image matrixes and labels
plots(imgs, titles=labels)          # <-- Plot Images with labels
#test_imgs = rgb_to_grayscale(imgs)  # <-- Convert RGB images to Grayscale ones by Tensorflow
#test_labels = labels

# Generating and Plot Image Data from Validation Set
VAL_BATCHES = ImageDataGenerator(rescale=NORMALIZER).\
    flow_from_directory(VALID_DIR,
    color_mode='grayscale',
    target_size=INPUT_DIM[0:2],
    classes=classes['VALIDATION'],
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE_VALID)

imgs, labels = next(VAL_BATCHES)   # <-- Extracting image matrixes and labels
plots(imgs, titles=labels)         # <-- Plot Images with labels
#val_imgs = rgb_to_grayscale(imgs)  # < -- Convert RGB images to Grayscale ones by Tensorflow
#val_labels = labels

In [ ]:
# Output of Generators
for data_batch, label_batch in TRAIN_BATCHES:
    print('data batch shape = {}'.format(data_batch.shape))
    print('labels batch shape = {}'.format(label_batch.shape))
    break

## CNN

In [ ]:
from keras import regularizers
# Build the CNN model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(INPUT_DIM)))
model.add(Activation('relu'))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(128, (4, 4))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (2, 2))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) 
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.02))) 
model.add(Activation('relu'))
model.add(Dense(3)) 
model.add(Activation('softmax'))
model.summary()

In [ ]:
# Define an optimizer for the model
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#opt = SGD(lr=0.01, decay=1e-6, momentum=0.85, nesterov=True)
#opt = RMSprop(lr=0.001, rho=0.8, epsilon=None, decay=0.0)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

In [ ]:
print('steps_per_epoch={}'.format(int(5215 / BATCH_SIZE_TRAIN)))
print('validation_steps={}'.format(int(624 / BATCH_SIZE_TEST)))

In [ ]:
h = model.fit_generator(
    TRAIN_BATCHES,
    steps_per_epoch=len(TRAIN_BATCHES),
    validation_data=TEST_BATCHES,
    validation_steps=len(TEST_BATCHES),
    epochs=35,
    verbose=2)
# Parameters meanings:
# steps_per_epoch = number_of_images / batch_size = 5215 / 64 = 82:
# --- Total number of steps (batches of samples) to yield from generator before declaring one 
#     epoch finished and starting the next epoch. It should typically be equal to the number 
#     of unique samples of your dataset divided by the batch size.
# Verbose:
# -- 0 (quiet): you just get the total numbers of tests executed and the global result
# -- 1 (default): you get the same plus a dot for every successful test or a F for every failure
# -- 2 (verbose): you get the help string of every test and the result

## Ploting learning curve

In [ ]:
# Plot Result Graph for Accuracy, Loss and Validation
def plot_model_result(model):
	'''
		-- model : Keras model.
	'''

	rcParams['figure.figsize'] = 14, 4 # Set plot size

	# Plot #1

	y1 = h.history['val_accuracy']
	y2 = h.history['accuracy']

	_ = plt.title('Model Results', family='Arial', fontsize=12)

	_ = plt.plot(y1, 
		color='blue', linewidth=1.5, marker='D', markersize=5,
		label='Validation acc.')
	_ = plt.plot(y2, 
		color='#9999FF', linewidth=1.5, marker='D', markersize=5,
		label='Training acc.')

	_ = plt.xlabel('Epochs', family='Arial', fontsize=10)
	_ = plt.ylabel('Score', family='Arial', fontsize=10)

	_ = plt.yticks(np.arange(0., 1.25, 0.1),
				   family='Arial', fontsize=10)

	if len(h.history['accuracy']) < 51:
		_ = plt.xticks(np.arange(0, len(h.history['accuracy']), 1),
					   family='Arial', fontsize=10)

	_ = plt.ylim((0., 1.))

	_ = plt.fill_between(np.arange(0, len(h.history['accuracy']), 1),
						 h.history['accuracy'], 0,
						 color = '#cccccc', alpha=0.5)

	_ = plt.grid(which='major', color='#cccccc', linewidth=0.5)
	_ = plt.legend(loc='best', shadow=True)
	_ = plt.margins(0.02)

	_ = plt.show()

	# Plot #2
	_ = plt.clf()

	_ = plt.plot(h.history['val_loss'], 
		color='red', linewidth=1.5, marker='D', markersize=5,
		label='Validation loss')
	_ = plt.plot(h.history['loss'], 
		color='#FF7F7F', linewidth=1.5, marker='D', markersize=5,
		label='Loss')

	_ = plt.xlabel('Epochs', family='Arial', fontsize=10)
	_ = plt.ylabel('Loss score', family='Arial', fontsize=10)

	if len(h.history['accuracy']) < 51:
		_ = plt.xticks(np.arange(0, len(h.history['accuracy']), 1),
					   family='Arial', fontsize=10)
	_ = plt.yticks(family='Arial', fontsize=10)

	_ = plt.grid(which='major', color='#cccccc', linewidth=0.5)
	_ = plt.legend(loc='best', shadow=True)
	_ = plt.margins(0.02)

	_ = plt.show()



In [ ]:
plot_model_result(h)

## Loading the model for further using 

In [ ]:
# Save the Model Weights
model.save_weights('model_100_eopchs_adam_20191030_01.h5')

# Save the Model to JSON
model_json = model.to_json()
with open('model_adam_20191030_01.json', 'w') as json_file:
    json_file.write(model_json)
    
print('Model saved to the disk.')

In [ ]:
# Load saved model and its weights
from keras.optimizers import Adam
from tensorflow.keras.models import model_from_json
from tensorflow.python.framework import ops
ops.reset_default_graph()
import h5py 
from PIL import Image
import PIL
from vb100_utils import *

In [ ]:
print('h5py version is {}'.format(h5py.__version__))

In [ ]:
# Get the architecture of CNN
json_file = open('model_adam.json')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# Get weights into the model
loaded_model.load_weights('model_100_eopchs_adam_20190807.h5')

In [ ]:
'''
Here I will simulate what will happen during deployment on a cloud.
Reading a given image, preparing it for CNN evaluation and make
a predictions with a returned class from a dictionary that has
been used for training.
'''

# Define optimizer and run
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
loaded_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
'''
Important Note! For this block optimizer is entered manualy as Tensorflow object.
For future, need to change it for include it as variable with full set of
parameters as Tensorflow variable.

'''
IMG = Image.open('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/val/BACTERIA/person1950_bacteria_4881.jpeg')
print(type(IMG))
IMG = IMG.resize((342, 257))
IMG = np.array(IMG)
print('po array = {}'.format(IMG.shape))
IMG = np.true_divide(IMG, 255)
IMG = IMG.reshape(1, 342, 257, 1)
print(type(IMG), IMG.shape)

predictions = loaded_model.predict(IMG)

print(loaded_model)
predictions_c = loaded_model.predict_classes(IMG)
print(predictions, predictions_c)

In [ ]:
classes = {'TRAIN': ['BACTERIA', 'NORMAL', 'VIRUS'],
           'VALIDATION': ['BACTERIA', 'NORMAL'],
           'TEST': ['BACTERIA', 'NORMAL', 'VIRUS']}

predicted_class = classes['TRAIN'][predictions_c[0]]
print('We think that is {}.'.format(predicted_class.lower()))

In [ ]:
# !pip install gevent

In [ ]:
# !pip install flask_ngrok

In [ ]:
MODEL_ARCHITECTURE = 'model_adam.json'
MODEL_WEIGHTS = 'model_100_eopchs_adam_20190807.h5'

# Load the model from external files
json_file = open(MODEL_ARCHITECTURE)
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# Get weights into the model
model.load_weights(MODEL_WEIGHTS)
# ::: MODEL FUNCTIONS :::
def model_predict(img_path, model):
	'''
		Args:
			-- img_path : an URL path where a given image is stored.
			-- model : a given Keras CNN model.
	'''
	IMG = image.load_img(img_path).convert('L')
	print(type(IMG))

	# Pre-processing the image
	IMG_ = IMG.resize((257, 342))
	print(type(IMG_))
	IMG_ = np.asarray(IMG_)
	print(IMG_.shape)
	IMG_ = np.true_divide(IMG_, 255)
	IMG_ = IMG_.reshape(1, 342, 257, 1)
	print(type(IMG_), IMG_.shape)

	print(model)

	model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
	prediction = model.predict_classes(IMG_)
	return prediction

In [ ]:
prediction = model_predict('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train/VIRUS/person1000_virus_1681.jpeg', model)

In [ ]:
predicted_class = classes['TRAIN'][prediction[0]]
predicted_class

## Flask

In [ ]:
# Import modules and packages
# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
from flask_ngrok import run_with_ngrok
# Import Keras dependencies
from tensorflow.keras.models import model_from_json
from tensorflow.python.framework import ops
ops.reset_default_graph()
from keras.preprocessing import image
# Import other dependecies
import numpy as np
import h5py
from PIL import Image
import PIL
import os
#Flask App Engine
# Define a Flask app
app = Flask(__name__)
run_with_ngrok(app)   
# ::: Prepare Keras Model :::
# Model files
MODEL_ARCHITECTURE = 'model_adam.json'
MODEL_WEIGHTS = 'model_100_eopchs_adam_20190807.h5'

# Load the model from external files
json_file = open(MODEL_ARCHITECTURE)
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# Get weights into the model
model.load_weights(MODEL_WEIGHTS)
# ::: MODEL FUNCTIONS :::
def model_predict(img_path, model):
	'''
		Args:
			-- img_path : an URL path where a given image is stored.
			-- model : a given Keras CNN model.
	'''
	IMG = image.load_img(img_path).convert('L')
	print(type(IMG))

	# Pre-processing the image
	IMG_ = IMG.resize((257, 342))
	print(type(IMG_))
	IMG_ = np.asarray(IMG_)
	print(IMG_.shape)
	IMG_ = np.true_divide(IMG_, 255)
	IMG_ = IMG_.reshape(1, 342, 257, 1)
	print(type(IMG_), IMG_.shape)

	print(model)

	model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
	prediction = model.predict_classes(IMG_)
	return prediction
# ::: FLASK ROUTES
@app.route('/', methods=['GET'])
def index():
	# Main Page
	return render_template('index.html')
@app.route('/predict', methods=['GET', 'POST'])
def upload():
	# Constants:
	classes = {'TRAIN': ['BACTERIA', 'NORMAL', 'VIRUS'],
	           'VALIDATION': ['BACTERIA', 'NORMAL'],
	           'TEST': ['BACTERIA', 'NORMAL', 'VIRUS']}

	if request.method == 'POST':

		# Get the file from post request
		f = request.files['file']

		# Save the file to ./uploads
		basepath = os.path.dirname(__file__)
		file_path = os.path.join(
			basepath, 'uploads', secure_filename(f.filename))
		f.save(file_path)

		# Make a prediction
		prediction = model_predict('Pneumonia-X-Rays-of-Human-Lungs-AI-project/data/train/VIRUS/person1000_virus_1681.jpeg', model)

		predicted_class = classes['TRAIN'][prediction[0]]
		print('We think that is {}.'.format(predicted_class.lower()))

		return str(predicted_class).lower()

if __name__ == '__main__':
	app.run()